# Exercise - Neural Networks using Keras

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 

Since you have a relatively small data set, I recommend using cross-validation to evaluate your accuracy.

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

## Checking for missing values

In [6]:
train.isna().sum()

BAD          0
LOAN         0
MORTDUE    151
VALUE       61
REASON      89
JOB         74
YOJ        156
DEROG      194
DELINQ     165
CLAGE      105
NINQ       153
CLNO        76
DEBTINC    556
dtype: int64

In [7]:
test.isna().sum()

BAD          0
LOAN         0
MORTDUE     71
VALUE       30
REASON      30
JOB         32
YOJ         60
DEROG       76
DELINQ      61
CLAGE       41
NINQ        59
CLNO        29
DEBTINC    262
dtype: int64

## Feature Engineering: Derive a new column

Examples:
- Ratio of delinquent to total number of credit lines
- Ratio of loan to value of current property
- Convert yr_renovated to a binary variable (i.e., renovated or not)
- (etc.)

In [8]:
#Ratio of delinquent to total number of credit lines

def new_col(df):
    
    df1 = df.copy()    
    df1['CLNO'] = np.where(df1['DELINQ'] > 0, 1, 0)
    return df1[['CLNO']]

In [9]:
new_col(train)

,CLNO
1552,0
2290,0
1398,0
1775,1
2299,0
...,...
1638,1
1095,1
1130,0
1294,0


##  Identify the numeric, binary, and categorical columns

In [10]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [11]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [12]:
categorical_columns

['REASON', 'JOB']

In [13]:
feat_eng_columns = ['DELINQ']

# Pipeline

In [14]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [15]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [16]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col))])

In [17]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('trans', my_new_column, feat_eng_columns),
        ],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [18]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  0.        ,
         0.        ,  0.        ],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        ,  0.        ],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        ,  0.        ]])

In [19]:
train_x.shape

(1750, 21)

# Tranform: transform() for TEST

In [20]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  1.        ,
         0.        ,  0.        ],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  1.        ],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        ,  0.        ]])

In [21]:
test_x.shape

(750, 21)

# Calculate the Baseline

In [22]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [23]:
from sklearn.metrics import accuracy_score

## Accuracy

In [24]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6034285714285714


In [25]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5773333333333334


# Train a shallow (one-layer) Keras model

In [26]:
#Define the model: for multi-class
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [27]:
#What is your input shape?
#(meaning: how many neurons should be in the input layer?)

train_x.shape

(1750, 21)

In [28]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=21))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(4, activation='softmax'))

#final layer: there has to be 4 nodes with softmax (because we have 4 categories)


In [29]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [30]:
# Fit the model

history = model.fit(train_x, train_target, 
                    validation_data=(test_x, test_target), 
                    epochs=20, batch_size=100)

Epoch 1/20
18/18 [==============================] - 1s 22ms/step - loss: 0.6866 - accuracy: 0.7120 - val_loss: 0.5658 - val_accuracy: 0.7347
Epoch 2/20
18/18 [==============================] - 0s 4ms/step - loss: 0.5162 - accuracy: 0.7520 - val_loss: 0.5386 - val_accuracy: 0.7253
Epoch 3/20
18/18 [==============================] - 0s 4ms/step - loss: 0.4834 - accuracy: 0.7623 - val_loss: 0.5008 - val_accuracy: 0.7573
Epoch 4/20
18/18 [==============================] - 0s 5ms/step - loss: 0.4724 - accuracy: 0.7737 - val_loss: 0.5046 - val_accuracy: 0.7427
Epoch 5/20
18/18 [==============================] - 0s 4ms/step - loss: 0.4674 - accuracy: 0.7811 - val_loss: 0.4925 - val_accuracy: 0.7520
Epoch 6/20
18/18 [==============================] - 0s 4ms/step - loss: 0.4517 - accuracy: 0.7869 - val_loss: 0.4866 - val_accuracy: 0.7627
Epoch 7/20
18/18 [==============================] - 0s 4ms/step - loss: 0.4485 - accuracy: 0.7909 - val_loss: 0.4949 - val_accuracy: 0.7613
Epoch 8/20
18/18 [=

In [31]:
# Train values

train_scores = model.evaluate(train_x, train_target, verbose=0)

train_scores

# In results, first is loss, second is accuracy

[0.3083086609840393, 0.8759999871253967]

In [32]:
# Print the values

print(f"Train {model.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {model.metrics_names[1]}: {train_scores[1]*100:.2f}%")


Train loss: 0.31
Train accuracy: 87.60%


In [33]:
# Test values

test_scores = model.evaluate(test_x, test_target, verbose=0)

test_scores

# In results, first is loss, second is accuracy

[0.42245984077453613, 0.8186666369438171]

In [34]:
# Print the values

print(f"Test {model.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {model.metrics_names[1]}: {test_scores[1]*100:.2f}%")


Test loss: 0.42
Test accuracy: 81.87%


# Train a deep (multi-layered) Keras model 

In [35]:
train_x.shape

(1750, 21)

In [48]:
#Define the model: for multi-class

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=21))
model.add(keras.layers.Dense(21, activation='relu'))
model.add(keras.layers.Dense(21, activation='relu'))
model.add(keras.layers.Dense(21, activation='relu'))
model.add(keras.layers.Dense(21, activation='softmax'))

#final layer: there has to be 4 nodes with softmax (because we have 4 categories)

In [54]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [55]:
# Fit the model

history = model.fit(train_x, train_target, 
                    validation_data=(test_x, test_target), 
                    epochs=20, batch_size=100)

Epoch 1/20
18/18 [==============================] - 1s 15ms/step - loss: 0.2911 - accuracy: 0.8800 - val_loss: 0.4845 - val_accuracy: 0.8147
Epoch 2/20
18/18 [==============================] - 0s 5ms/step - loss: 0.2447 - accuracy: 0.8966 - val_loss: 0.4590 - val_accuracy: 0.8040
Epoch 3/20
18/18 [==============================] - 0s 4ms/step - loss: 0.2351 - accuracy: 0.9057 - val_loss: 0.4785 - val_accuracy: 0.8067
Epoch 4/20
18/18 [==============================] - 0s 5ms/step - loss: 0.2344 - accuracy: 0.9057 - val_loss: 0.4836 - val_accuracy: 0.8120
Epoch 5/20
18/18 [==============================] - 0s 4ms/step - loss: 0.2328 - accuracy: 0.9034 - val_loss: 0.4639 - val_accuracy: 0.8160
Epoch 6/20
18/18 [==============================] - 0s 4ms/step - loss: 0.2471 - accuracy: 0.8949 - val_loss: 0.4940 - val_accuracy: 0.8160
Epoch 7/20
18/18 [==============================] - 0s 4ms/step - loss: 0.2339 - accuracy: 0.9103 - val_loss: 0.5476 - val_accuracy: 0.8187
Epoch 8/20
18/18 [=

In [56]:
# Train values

train_scores = model.evaluate(train_x, train_target, verbose=0)

train_scores

# In results, first is loss, second is accuracy

[0.1854034811258316, 0.9217143058776855]

In [57]:
# Print the values

print(f"Train {model.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {model.metrics_names[1]}: {train_scores[1]*100:.2f}%")


Train loss: 0.19
Train accuracy: 92.17%


In [58]:
# Test values

test_scores = model.evaluate(test_x, test_target, verbose=0)

test_scores

# In results, first is loss, second is accuracy

[0.6677367091178894, 0.8293333053588867]

In [59]:
# Print the values

print(f"Test {model.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {model.metrics_names[1]}: {test_scores[1]*100:.2f}%")


Test loss: 0.67
Test accuracy: 82.93%


# Optional: try different activation functions, optimizers, or configurations (such as wide and deep) to build other models